In [1]:
using NBInclude
@nbinclude("datastructs.ipynb")
@nbinclude("helpers.ipynb")

Genome(Chromosome[Chromosome(AbstractGene[Gene
  id: Int64 1
  dna: Char 'a'
  reverse: Bool false
, Gene
  id: Int64 2
  dna: Char 'b'
  reverse: Bool false
, Gene
  id: Int64 3
  dna: Char 'c'
  reverse: Bool false
])], Dict(2 => 'b', 3 => 'c', 1 => 'a'), Dict('a' => 1, 'c' => 3, 'b' => 2))SRC ADJ LIST
(.,a:t)(a:h,b:t)(b:h,c:t)(c:h,.)
SRC ADJ SET
(b:h,c:t)(c:h,.)(.,a:t)(a:h,b:t)
|
v
TARGET ADJ LIST
(a:h,a:t)(a:h,b:t)(b:h,c:t)(c:h,a:t)
TARGET ADJ SET
(a:h,a:t)(a:h,b:t)(b:h,c:t)(c:h,a:t)Genome(Chromosome[Chromosome(AbstractGene[Gene
  id: Int64 1
  dna: Char 'a'
  reverse: Bool false
, Gene
  id: Int64 2
  dna: Char 'b'
  reverse: Bool false
, Gene
  id: Int64 3
  dna: Char 'c'
  reverse: Bool false
])], Dict(2 => 'b', 3 => 'c', 1 => 'a'), Dict('a' => 1, 'c' => 3, 'b' => 2))SRC ADJ LIST
(.,a:t)(a:h,b:t)(b:h,c:t)(c:h,.)
SRC ADJ SET
(c:h,.)(.,a:t)(a:h,b:t)(b:h,c:t)
|
v
TARGET ADJ LIST
(a:h,a:t)(a:h,b:t)(b:h,c:t)(c:h,a:t)
TARGET ADJ SET
(a:h,b:t)(a:h,a:t)(c:h,a:t)(b:h,c:t)

combine_ge (generic function with 1 method)

In [2]:
function reassign_locs(u_idx::Int, v_idx::Int, pq::Adjacency, excluding_pq::Adjacency, gid_to_l::DefaultDict{Int, Vector{Int}})
    # want to update dictionary given: 
    # src_adjs[u_idx] = pq 
    # src_adjs[v_idx] = excluding_pq 

    idx = Ref{Int}(u_idx)
    if pq.left.gene != Telomere() 
        assign_ge_idx_to_gid_to_locdict(pq.left, idx, gid_to_l)
    end 
    if pq.right != Telomere() 
        assign_ge_idx_to_gid_to_locdict(pq.right, idx, gid_to_l)
    end 
    
    idx = Ref{Int}(v_idx)
    if excluding_pq.left.gene != Telomere() 
        assign_ge_idx_to_gid_to_locdict(excluding_pq.left, idx, gid_to_l)
    end 
    if excluding_pq.right.gene != Telomere() 
        assign_ge_idx_to_gid_to_locdict(excluding_pq.right, idx, gid_to_l) 
    end 
end 

function update_documentation(u_idx::Int, v_idx::Int, pq::Adjacency, excluding_pq::Adjacency, src_adjs::Vector{Adjacency}, src_gid_to_l::DefaultDict{Int, Vector{Int}})
    # update src adjs
    if u_idx == 0
        u_idx = length(src_adjs)+1
        push!(src_adjs, pq)
    end
    if v_idx == 0
        v_idx = length(src_adjs)+1
        push!(src_adjs, excluding_pq)
    end 
    
    src_adjs[u_idx] = pq 
    src_adjs[v_idx] = excluding_pq 

    # update geneend locations  
    reassign_locs(u_idx, v_idx, pq, excluding_pq, src_gid_to_l)
end 


update_documentation (generic function with 1 method)

In [3]:
# find dcj operations and distance
function find_dcj_dist_ops(src_adjs::Vector{Adjacency}, tar_adj_list::Vector{Adjacency}, src_gid_to_l::DefaultDict{Int, Vector{Int}}, tar_gid_to_l::DefaultDict{Int, Vector{Int}}, src_adj_set::Set{Adjacency}, target_adj_set::Set{Adjacency})
    count = 0  
    telomere_idxs = Vector{Int}()
    updated_adj_list = src_adjs

    print("indexing through target adj list...")
    
    
    # for each adj {p, q} in target genome 
    for (i, adj) in pairs(tar_adj_list) 
        p = adj.left  
        q = adj.right  
        if p.gene == Telomere() || q.gene == Telomere()  # telomeres handled in next loop
            push!(telomere_idxs, i)  
        else 
            print("\n\n")
            show(updated_adj_list)
            
            print("\nADJ ", i, " :::: ")
            show(adj, false)
            print("\n")

            #  let u be ge of adj from genome A that == p.gene 
            #  let v be ge of adj from genome A that == q.gene 
            u, u_idx, u_adj = find_tar_ge_in_src_adjs(p, src_gid_to_l, tar_gid_to_l, src_adjs)  
            v, v_idx, v_adj = find_tar_ge_in_src_adjs(q, src_gid_to_l, tar_gid_to_l, src_adjs)  
            
            #  if u != v, replace u and v in A by {p, q} and (u\{p}) U (v\{q})
            if u_idx != v_idx  
                print("prev adj list ::: ")
                show(convert_adj_set_as_adj_list(src_adj_set), u_adj, v_adj)
 
                if u.gene == Telomere() && v.gene == Telomere() 
                    u_partner = non_telo_end(u_adj)
                    v_partner = non_telo_end(v_adj)
                    pq = combine_ge(u_partner, u_idx, v_partner, v_idx)
                    two_telos = Adjacency(GeneEnd(Telomere()), GeneEnd(Telomere()))

                    #  remove adj with u, v in src_adj_set 
                    delete!(src_adj_set, src_adjs[u_idx])
                    delete!(src_adj_set, src_adjs[v_idx])

                    #  add pq to src_adj_set (not two-telomere adjacency)
                    push!(src_adj_set, pq)
                    push!(src_adj_set, two_telos)              

                    print("\nupdated adj list ::: ")
                    updated_adj_list = convert_adj_set_as_adj_list(src_adj_set)
                    show(updated_adj_list, pq, two_telos)

                    update_documentation(u_idx, v_idx, pq, two_telos, src_adjs, src_gid_to_l)
                    
                    # println("\n\nupdated src_adjs")
                    # show(src_adjs)
                    # println("\nupdated src_gid_to_location")
                    # show(src_gid_to_l, id_to_char)
                    
                elseif u.gene == Telomere() || v.gene == Telomere() 
                    # print("[[ u or v  is a telomere ]]") 
                    pq = combine_ge(p, u_idx, q, v_idx)
                    if u.gene == Telomere() 
                        other_ge_v = other_adjacency_end(v, src_adjs[v_idx])
                        one_telo = combine_ge(GeneEnd(Telomere), u_idx, other_ge_v, v_idx)
                    else 
                        other_ge_u = other_adjacency_end(u, src_adjs[u_idx])
                        one_telo = combine_ge(other_ge_u, u_idx, GeneEnd(Telomere()), v_idx)
                    end 

                     #  remove adj with u, v in src_adj_set 
                    delete!(src_adj_set, src_adjs[u_idx])
                    delete!(src_adj_set, src_adjs[v_idx])
                    
                    #  add adjusted adjacencies to src_adj_set
                    push!(src_adj_set, pq)
                    push!(src_adj_set, one_telo)
                    
                    #  print("updated adj set ::: ")
                    # show(src_adj_set)
                    print("\nupdated adj list ::: ")
                    updated_adj_list = convert_adj_set_as_adj_list(src_adj_set)
                    show(updated_adj_list, pq, one_telo)

                    update_documentation(u_idx, v_idx, pq, one_telo, src_adjs, src_gid_to_l)
                    
                    # println("\n\nupdated src_adjs")
                    # show(src_adjs)
                    # println("\nupdated src_gid_to_location")
                    # show(src_gid_to_l, id_to_char)

                else # both are adj with two gene ends (no telomeres)   
                    pq, excluding_pq = update_adj_set(p, q, u, v, u_idx, v_idx, src_adjs, src_adj_set)
                    
                    # print("updated adj set ::: ")
                    # show(src_adj_set)
                    print("\nupdated adj list ::: ")
                    updated_adj_list = convert_adj_set_as_adj_list(src_adj_set)
                    show(updated_adj_list, pq, excluding_pq)

                    update_documentation(u_idx, v_idx, pq, excluding_pq, src_adjs, src_gid_to_l)
                    
                    # println("\n\nupdated src_adjs")
                    # show(src_adjs)
                    # println("\nupdated src_gid_to_location")
                    # show(src_gid_to_l, id_to_char)
                end
                count += 1 
            end   
        end
    end 

    # for each telomere {p} in genome B 
    for idx in telomere_idxs
        print("\n\n")
        show(updated_adj_list)
        
        telo_adj = tar_adj_list[idx]
        non_telo_ge = other_adjacency_end(GeneEnd(Telomere()), telo_adj)
        
        print("\nADJ ", idx, " :::: ")
        show(telo_adj, false)
        print("\n")

        #  let u be ge of genome A that contains p 
        u, u_idx, u_adj = find_tar_ge_in_src_adjs(non_telo_ge, src_gid_to_l, tar_gid_to_l, src_adjs)  

        #  if u is an adjacency, then replace u in A by {p} and (u\{p}) 
        if u.gene != Telomere()
            p = Adjacency(u, GeneEnd(Telomere()))
            excluding_p = Adjacency(other_adjacency_end(u, u_adj), GeneEnd(Telomere()))

            #  remove adj with u, v in src_adj_set 
            delete!(src_adj_set, src_adjs[u_idx])

            #  add p, excluding_p to src_adj_set 
            push!(src_adj_set, p)
            push!(src_adj_set, excluding_p)   

            # print("updated adj set ::: ")
            # show(src_adj_set)
            print("\nupdated adj list ::: ")
            updated_adj_list = convert_adj_set_as_adj_list(src_adj_set)
            show(updated_adj_list, p, excluding_p)

            update_documentation(u_idx, 0, p, excluding_p, src_adjs, src_gid_to_l)
            
            # println("\n\nupdated src_adjs")
            # show(src_adjs)
            # println("\nupdated src_gid_to_location")
            # show(src_gid_to_l, id_to_char)

            count += 1 
        end 
    end 

    return count 
end 

src = ".abcdefg."
target = ".abcdGFE." 

calculate_distance(src, target)

Genome(Chromosome[Chromosome(AbstractGene[Telomere(), Gene
  id: Int64 1
  dna: Char 'a'
  reverse: Bool false
, Gene
  id: Int64 2
  dna: Char 'b'
  reverse: Bool false
, Gene
  id: Int64 3
  dna: Char 'c'
  reverse: Bool false
, Gene
  id: Int64 4
  dna: Char 'd'
  reverse: Bool false
, Gene
  id: Int64 5
  dna: Char 'g'
  reverse: Bool true
, Gene
  id: Int64 6
  dna: Char 'f'
  reverse: Bool true
, Gene
  id: Int64 7
  dna: Char 'e'
  reverse: Bool true
, Telomere()])], Dict(5 => 'G', 4 => 'd', 6 => 'F', 7 => 'E', 2 => 'b', 3 => 'c', 1 => 'a'), Dict('f' => 6, 'g' => 5, 'a' => 1, 'c' => 3, 'd' => 4, 'e' => 7, 'b' => 2))SRC ADJ LIST
(.,a:t)(a:h,b:t)(b:h,c:t)(c:h,d:t)(d:h,e:t)(e:h,f:t)(f:h,g:t)(g:h,.)
SRC ADJ SET
(d:h,e:t)(.,a:t)(a:h,b:t)(g:h,.)(b:h,c:t)(e:h,f:t)(f:h,g:t)(c:h,d:t)
|
v
TARGET ADJ LIST
(.,a:t)(a:h,b:t)(b:h,c:t)(c:h,d:t)(d:h,g:h)(g:t,f:h)(f:t,e:h)(e:t,.)
TARGET ADJ SET
(.,a:t)(b:h,c:t)(a:h,b:t)(c:h,d:t)(d:h,g:h)(g:t,f:h)(f:t,e:h)(e:t,.)

In [4]:
# permutations that require max number of dcj operations  - diameter of an instance 

# dcj1 -- (a,b) (c,d) --> (a, c) (b, d)

# find the largest value of n which you can find the diameter 
# n --> diameter (randomly sample x number)
# increase n  

#     max and min distance maps distance 


#     mirjalil - python library on nature-inspired algos 
#     whale optimization algorithm (bubble net?) - populat and simple  
#     gray wolf optimization population algorithm
#     sim to 
#     differential evolution (not from mirjalil tho; it's older ) 